In [1]:
import os, time, itertools, imageio, pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

I:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#hyperparameters
batch_size = 128
lr = 0.0002
train_epoch = 50
nb_classes = 10


# load MNIST
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
train_set = (mnist.train.images) 
#add normalization
train_label = mnist.train.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def eval(label = 0):
    _z = np.random.rand(1,100)
    targets = np.array([[label]]).reshape(-1)
    one_hot_targets = np.eye(nb_classes)[targets]
    pop = sess.run(G_z, {z: _z, y: one_hot_targets, isTrain: False})
    return pop.reshape(28,28)

In [4]:
def makeGif(filename = 'testing.gif', fps = 5):
    data = []
    for i in range(10):
        x = eval(i)
        data.append(x)
    imageio.mimsave(filename, data, fps=fps)


In [5]:
def generator(x, y, isTrain=True, reuse=False):
    with tf.variable_scope('gen', reuse=reuse):
        w_init = tf.contrib.layers.xavier_initializer()

        cat1 = tf.concat([x, y], 1)

        dense1 = tf.layers.dense(cat1, 128, kernel_initializer=w_init)
        relu1 = tf.nn.relu(dense1)

        dense2 = tf.layers.dense(relu1, 784, kernel_initializer=w_init)
        o = tf.nn.tanh(dense2)

        return o

In [6]:
def discriminator(x, y, isTrain=True, reuse=False):
    with tf.variable_scope('dis', reuse=reuse):
        w_init = tf.contrib.layers.xavier_initializer()

        cat1 = tf.concat([x, y], 1)

        dense1 = tf.layers.dense(cat1, 128, kernel_initializer=w_init)
        lrelu1 = tf.nn.relu(dense1)

        dense2 = tf.layers.dense(lrelu1, 1, kernel_initializer=w_init)
        o = tf.nn.sigmoid(dense2)

        return o, dense2

In [7]:
onehot = np.eye(10)

x = tf.placeholder(tf.float32, shape=(None, 784))
y = tf.placeholder(tf.float32, shape=(None, 10))
z = tf.placeholder(tf.float32, shape=(None, 100))
isTrain = tf.placeholder(dtype=tf.bool)

G_z = generator(z, y, isTrain)

D_real, D_real_logits = discriminator(x, y, isTrain)
D_fake, D_fake_logits = discriminator(G_z, y, isTrain, reuse=True)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones([batch_size, 1])))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros([batch_size, 1])))
D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones([batch_size, 1])))

T_vars = tf.trainable_variables()
D_vars = [var for var in T_vars if var.name.startswith('dis')]
G_vars = [var for var in T_vars if var.name.startswith('gen')]

In [8]:
with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
    D_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(D_loss, var_list=D_vars)
    G_optim = tf.train.AdamOptimizer(lr, beta1=0.5).minimize(G_loss, var_list=G_vars)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [ ]:
start_time = time.time()
for epoch in range(train_epoch):
    G_losses = []
    D_losses = []
    epoch_start_time = time.time()
    for iter in range(len(train_set) // batch_size):
        # update discriminator
        x_ = train_set[iter * batch_size:(iter + 1) * batch_size]
        y_ = train_label[iter * batch_size:(iter + 1) * batch_size]

        z_ = np.random.normal(0, 1, (batch_size, 100))

        loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, y: y_, z: z_, isTrain: True})
        D_losses.append(loss_d_)

        # update generator
        z_ = np.random.normal(0, 1, (batch_size, 100))
        y_ = np.random.randint(0, 9, (batch_size, 1))
        y_ = onehot[y_.astype(np.int32)].squeeze()
        loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, x: x_, y: y_, isTrain: True})
        G_losses.append(loss_g_)

    print('[%d/%d] - loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, np.mean(D_losses), np.mean(G_losses)))

[1/50] - loss_d: 1.280, loss_g: 0.882
[2/50] - loss_d: 1.035, loss_g: 0.993
[3/50] - loss_d: 1.024, loss_g: 1.108
[4/50] - loss_d: 1.042, loss_g: 1.130
[5/50] - loss_d: 1.007, loss_g: 1.204
[6/50] - loss_d: 0.935, loss_g: 1.286
[7/50] - loss_d: 0.896, loss_g: 1.330
[8/50] - loss_d: 0.951, loss_g: 1.201
[9/50] - loss_d: 1.053, loss_g: 1.097
[10/50] - loss_d: 1.068, loss_g: 1.116
[11/50] - loss_d: 1.078, loss_g: 1.111
[12/50] - loss_d: 1.063, loss_g: 1.107
[13/50] - loss_d: 1.070, loss_g: 1.109
[14/50] - loss_d: 1.081, loss_g: 1.126
[15/50] - loss_d: 1.030, loss_g: 1.105
[16/50] - loss_d: 0.995, loss_g: 1.120
[17/50] - loss_d: 0.998, loss_g: 1.156
[18/50] - loss_d: 0.983, loss_g: 1.208
[19/50] - loss_d: 0.955, loss_g: 1.216
[20/50] - loss_d: 0.935, loss_g: 1.271
[21/50] - loss_d: 0.911, loss_g: 1.303
[22/50] - loss_d: 0.901, loss_g: 1.287
[23/50] - loss_d: 0.899, loss_g: 1.280
[24/50] - loss_d: 0.894, loss_g: 1.283
[25/50] - loss_d: 0.881, loss_g: 1.350
[26/50] - loss_d: 0.878, loss_g: 1

In [ ]:
makeGif('Results.gif')
plt.imsave('Result_1.png', eval(1))
